In [76]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [77]:
df=pd.read_csv(r'C:\Users\USER\Downloads\marketing_campaign (3).csv')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,4/9/2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,8/3/2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21/08/2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10/2/2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19/01/2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [78]:
def Data_Cleaning(df):
    initial_shape = df.shape
    print(f'Initial DataFrame shape: {initial_shape}')
    
    if df.duplicated().sum() > 0:
        df = df.drop_duplicates()
        print('Deleting duplicated rows')
    else:
        print("NO DUPLICATED ROWS!")

    for c in df.columns:
        if df[c].isnull().sum() / len(df) > 0.8:
            print('Deleting column', c, 'because of high ratio of nulls' )
            del df[c]
            
        elif df[c].nunique() == 1:
            print('Deleting column', c, 'because of single value')
            del df[c]
            
        elif len(df[df[c] == df[c].mode()[0]]) / len(df) > 0.8:
            print('Deleting column', c, 'because of imbalanced value')
            del df[c]
    for c in df.select_dtypes(include=['object']):
        if df[c].nunique()/len(df)>0.8:
            print('Deleting column', c, 'because of high ratio of unique values')
            del df[c]
    final_shape = df.shape
    print(f'Final DataFrame shape: {final_shape}')
    return df

df=df
df = Data_Cleaning(df)

Initial DataFrame shape: (2240, 29)
NO DUPLICATED ROWS!
Deleting column AcceptedCmp3 because of imbalanced value
Deleting column AcceptedCmp4 because of imbalanced value
Deleting column AcceptedCmp5 because of imbalanced value
Deleting column AcceptedCmp1 because of imbalanced value
Deleting column AcceptedCmp2 because of imbalanced value
Deleting column Complain because of imbalanced value
Deleting column Z_CostContact because of single value
Deleting column Z_Revenue because of single value
Deleting column Response because of imbalanced value
Final DataFrame shape: (2240, 20)


In [79]:
del df['ID']

In [80]:
df.dtypes

Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumCatalogPurchases      int64
NumStorePurchases        int64
NumWebVisitsMonth        int64
dtype: object

In [81]:
for c in df.select_dtypes(include=['object']).columns:
    print(df[c].value_counts())
    

Education
Graduation    1127
PhD            486
Master         370
2n Cycle       203
Basic           54
Name: count, dtype: int64
Marital_Status
Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: count, dtype: int64
Dt_Customer
31/08/2012    12
12/9/2012     11
14/02/2013    11
12/5/2014     11
20/08/2013    10
              ..
5/8/2012       1
18/11/2012     1
25/05/2013     1
14/04/2013     1
9/1/2014       1
Name: count, Length: 663, dtype: int64


In [82]:
df= df[~df['Marital_Status'].isin(['Alone', 'Absurd', 'YOLO'])]

In [83]:
df['Education']=df['Education'].map({'Graduation':5,'PhD':4,'Master':3,'2n Cycle':2,'Basic':1})

In [84]:
dummies=pd.get_dummies(df['Marital_Status'])
dummies=dummies.astype(int)
df=pd.concat([df,dummies],axis=1)
df=df.drop(columns=['Marital_Status'])

In [85]:
df['Income']=df['Income'].fillna(df['Income'].mean())

In [86]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d/%m/%Y', errors='coerce')

In [87]:
df['Day'] = df['Dt_Customer'].dt.day
df['Month'] = df['Dt_Customer'].dt.month
df['Year'] = df['Dt_Customer'].dt.year

In [88]:
del df['Dt_Customer']

In [89]:
df.head()

,Year_Birth,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumStorePurchases,NumWebVisitsMonth,Divorced,Married,Single,Together,Widow,Day,Month,Year
0,1957,5,58138.0,0,0,58,635,88,546,172,...,4,7,0,0,1,0,0,4,9,2012
1,1954,5,46344.0,1,1,38,11,1,6,2,...,2,5,0,0,1,0,0,8,3,2014
2,1965,5,71613.0,0,0,26,426,49,127,111,...,10,4,0,0,0,1,0,21,8,2013
3,1984,5,26646.0,1,0,26,11,4,20,10,...,4,6,0,0,0,1,0,10,2,2014
4,1981,4,58293.0,1,0,94,173,43,118,46,...,6,5,0,1,0,0,0,19,1,2014


In [90]:
for c in df.select_dtypes(exclude = ['object']):
    if df[c].nunique()>2:
	    df[c] = (df[c] - df[c].min()) / (df[c].max() - df[c].min())

In [92]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=3, random_state=0, n_init=10)  
kmeans.fit(df)

# Küme merkezlerini DataFrame'e aktarma
cols = df.columns
results = pd.DataFrame(kmeans.cluster_centers_, columns=cols)

print("Cluster Centers:")
print(results)

Cluster Centers:
   Year_Birth  Education    Income   Kidhome  Teenhome   Recency  MntWines  \
0    0.725695   0.760345  0.077456  0.225000  0.264655  0.506130  0.205510   
1    0.734899   0.779151  0.075722  0.213561  0.241445  0.499648  0.205604   
2    0.743494   0.771123  0.075194  0.228009  0.255787  0.487654  0.200590   

   MntFruits  MntMeatProducts  MntFishProducts  ...  NumStorePurchases  \
0   0.127387         0.097451         0.150546  ...           0.441247   
1   0.138793         0.100538         0.148915  ...           0.443697   
2   0.129321         0.093149         0.136605  ...           0.450053   

   NumWebVisitsMonth      Divorced       Married        Single      Together  \
0           0.263017  6.938894e-17 -2.775558e-16 -2.775558e-16  1.000000e+00   
1           0.265209  2.940431e-01 -2.775558e-16  6.083650e-01  5.551115e-17   
2           0.268229  1.249001e-16  1.000000e+00 -1.942890e-16  1.110223e-16   

          Widow       Day     Month      Year  
0  2